In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pathlib import Path
import json

extracted_answers = []
query_file = "queries.json"


dotenv_path = Path('../global_environment.env')
load_dotenv(dotenv_path=dotenv_path)
openai_key = os.getenv('OPENAI_KEY')

client = OpenAI(
        api_key=openai_key
)

if(openai_key):
    print('Key loaded successfully')
else:
    print('API Key not found')

Key loaded successfully


In [10]:
def extract_summary(descriptions, desctype):
    with open(query_file, 'r') as f:
        queries = json.load(f)
    
    description_list = "\n".join(descriptions)

    if desctype == 'sensory':
        query = queries['summaryQuerySensory']
    elif desctype == 'emotional':
        query = queries['summaryQueryEmotional']
    elif desctype == 'associative':
        query = queries['summaryQueryAssociative']
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": queries['summaryAgentInstructions']},
                {"role": "user", "content": query+"\n"+description_list}
            ]
        )
        print(response.choices[0].message)
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error extracting keywords: {e}")
        return "Error" 

In [ ]:
def summarize_signal(signal_obj):
    sensory_descriptions = signal_obj['sensory_descriptions']
    emotional_descriptions = signal_obj['emotional_descriptions']
    associative_descriptions = signal_obj['associative_descriptions']
    sensory_summary = extract_summary(sensory_descriptions, 'sensory')
    emotional_summary = extract_summary(emotional_descriptions, 'emotional')
    associative_summary = extract_summary(associative_descriptions, 'associative')
    signal_obj['sensory_summary'] = sensory_summary
    signal_obj['emotional_summary'] = emotional_summary
    signal_obj['associative_summary'] = associative_summary
    return signal_obj

def main():
    with open('collated_descriptions.json', 'r') as f:
        haptic_signals = json.load(f)
    signal_output = []
    for signal_index in range(len(haptic_signals)):
        signal_output.append(summarize_signal(haptic_signals[signal_index]))
    print("Writing output to signal_summaries_concise.json")
    with open('signal_summaries.json', 'w') as f:
        json.dump(signal_output, f, indent=4)

if __name__ == "__main__":
    main()

ChatCompletionMessage(content='The summarized descriptions of the haptic sensation depict it primarily as a slow, rhythmic heartbeat. Many comparisons are made to a heartbeat, with references to the pacing and structure of beats and pauses. Additionally, some descriptions liken the sensation to a phone vibrating, with mentions of a buzzing or bumpy feeling. Overall, the sensations convey a combination of slow rhythmic beats, buzzing notifications, and a textured experience.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
ChatCompletionMessage(content='The descriptions of haptic sensations reveal a range of emotions. Some individuals experience anxiety, with feelings of unease and alertness associated with the sensation. Others report positive emotions, feeling calm, comforted, or active, with one describing the sensation as enjoyable and fuzzy. A reassuring feeling is noted, while one person also mentions a negative sensation described as greasy. Over